In [ ]:
%run my_import

#uri = socket.gethostname()+".local:5555"
#uri = "localhost:5555"
#uri = "nuc-i7.local:5555"
uri = "com-exp.local:5555"
#uri = "advantech.local:5555"
#uri = "192.168.1.100:5555"

io = set_uri(uri)
io.debug = False
scan_ids = list(reply_cmd(master_cmd_get_slave_descr).keys())
print('Found ESC ids {}'.format(scan_ids))


In [ ]:
from simple_pid import PID

ch_map = {
    0: 'force_x',
    1: 'force_y',
    2: 'force_z',
    3: 'torque_x',
    4: 'torque_y',
    5: 'torque_z',
}

def read_channel(bid:int, ch:int)->dict :
    write_sdo({'adc_idx':ch},[bid])
    reply_cmd(ctrl_cmd_get_adc.set_bid(bid))
    return read_sdo(['adc_conv','adc_raw'],[bid])[bid]

def read_channel_dac_offset(bid:int, ch:int, offset:int) :
    write_sdo({'adc_idx': ch,'dac_offset':offset},[bid])
    reply_cmd(ctrl_cmd_set_dac.set_bid(bid))
    reply_cmd(ctrl_cmd_get_adc.set_bid(bid))
    return read_sdo(['adc_conv','adc_raw'],[bid])[bid]

def read_channel_adc_offset(bid:int, ch:int, offset:float) :
    write_sdo({'adc_idx': ch,'adc_conv_offset':offset},[bid])
    # call Set_adc_offset_flash :
    #  - sdo.flash.adc_conv_offs[sdo.ram.adc_idx] = sdo.ram.adc_conv_offset;
    ctrl_status_cmd(0x00AF, bid)
    reply_cmd(ctrl_cmd_get_adc.set_bid(bid))
    return read_sdo([ch_map[ch],'adc_conv','adc_raw'],[bid])[bid]

def reset_DAC() :
    for ch in range(6) :
        ret = read_channel_offset(iD,ch,0x7FFF)
        print(ret)

def tune_adc_chan(bid:int, adc_idx:int) :
    dac_offset = 0x7FFF
    hister = 5
    tuned = False
    while ( not tuned ):
        yaml_msg = read_channel_dac_offset(bid,adc_idx,dac_offset)
        adc_raw = yaml_msg['adc_raw']
        adc_conv = yaml_msg['adc_conv']
        if ( adc_raw > 0x1FFF+hister ) :
            if ( dac_offset < 0xFFFF ) :
                # old pcb see github ticket #56
                #dac_offset += 2
                dac_offset -= 2
        elif ( adc_raw < 0x1FFF-hister ) :
            if ( dac_offset > 0 ) :
                # old pcb see github ticket #56
                #dac_offset -= 2
                dac_offset += 2
        else : 
            tuned = True
        time.sleep(0.001)
        yield(tuned, adc_raw, adc_conv, dac_offset)
        
def tune_adc_offset(bid:int, adc_idx:int, set_point:float) :
    adc_offset = 0.0
    hister = 0.05
    tuned = False
    while ( not tuned ):
        yaml_msg = read_channel_adc_offset(bid,adc_idx,adc_offset)
        adc_raw = yaml_msg['adc_raw']
        adc_conv = yaml_msg['adc_conv']
        adc = yaml_msg[ch_map[adc_idx]]
        adc_offset = adc_conv
        if abs(adc) < hister :
            tuned = True
        time.sleep(0.001)
        yield(tuned, adc_raw, adc_conv, adc_offset)

In [ ]:
iD = scan_ids[0]
sdo_objd = reply_cmd(SdoInfo(u'SDO_OBJD').set_bid(iD))
sdo_names = reply_cmd(SdoInfo(u'SDO_NAME').set_bid(iD))
sdo_dict = reply_cmd(SdoCmd(rd_sdo=sdo_names,wr_sdo={}).set_bid(iD))
sdo_dict

In [ ]:
flash_param = ['fw_ver','analog_sample_freq_HZ','dac_tune_bitflag',
               'smpl_num_pow2','dac_tune_hysteresis','adc_conv_coeff']
sdo_filter(flash_param, sdo_dict)

In [ ]:
read_sdo(['smpl_num_pow2'],[iD])

In [ ]:
write_sdo({'smpl_num_pow2': 1},[iD])

In [ ]:

write_sdo({'adc_conv_coeff': 1.055},[iD])
reply_cmd(flash_cmd_save_flash.set_bid(iD))

## Load default calib matrix

In [ ]:
ctrl_status_cmd_str('LOAD_DFLT_CM', iD)

## read 1 channel

In [ ]:
read_channel(iD,0)

In [ ]:
read_channel_offset(iD,0,0x7FFF)

In [ ]:
read_channel_adc_offset(iD,0,-4)

## read all 6 channels once

In [ ]:

adc_conv = list()
adc_raw = list()
for j in range(1):
    for i in range(6):
        yaml_msg = read_channel(iD,i)
        adc_raw.append(yaml_msg['adc_raw'])
        adc_conv.append(yaml_msg['adc_conv'])
        print(yaml_msg)
        time.sleep(0.1)
        
[hex(x) for x in adc_raw]

## reset DAC 

In [ ]:
reset_DAC()

## Set all analog channel to Zero using DAC

In [ ]:
for ch in range(6) :
    print("--- {0} ---".format(ch))
    for tuned, adc_raw, adc_conv, offset in tune_adc_chan(iD,ch) :
        print("\r%d %f 0x%X 0x%X"%(tuned, adc_conv, adc_raw, offset), end="")
        if tuned :
            write_sdo({'adc_idx': ch,'dac_offset':offset},[iD])
            reply_cmd(ctrl_cmd_set_dac_flash.set_bid(iD))
    print()


## Reset ADC offset

In [ ]:
for ch in range(6) :
    read_channel_adc_offset(iD,ch,0)

In [ ]:
for ch in range(6) :
    print("--- {0} ---".format(ch))
    for tuned, adc_raw, adc_conv, offset in tune_adc_offset(iD,ch,0) :
        print("\r%d %f 0x%X %f"%(tuned, adc_conv, adc_raw, offset), end="")
        if tuned :
            pass
            #write_sdo({'adc_idx': ch,'dac_offset':offset},[iD])
            #reply_cmd(ctrl_cmd_set_dac_flash.set_bid(iD))
    print()


In [ ]:
write_sdo({'dac_tune_bitflag': 0x3F},[iD])
reply_cmd(flash_cmd_save_flash.set_bid(iD))

In [ ]:
write_sdo({'smpl_num_pow2': 8},[iD])
reply_cmd(flash_cmd_save_flash.set_bid(iD))

In [ ]:
cnt = 10
dac_offset = 0x7FFF
while (cnt) :
    cnt -= 1
    print(read_channel_offset(iD,0,dac_offset))
    dac_offset += 10


In [ ]:
ctrl_status_cmd(0x0012, iD)

In [ ]:
ctrl_status_cmd(0x0021, iD)

In [ ]:
ctrl_status_cmd(0x00AF, iD)

In [ ]:
reply_cmd(flash_cmd_load_default.set_bid(696))

In [ ]:
ctrl_status_cmd_str('LOAD_CM', iD)